1.       資料前處理


         1.1      讀取 csv 檔後取前 1 萬筆資料，僅保留 "Text"、"Score" 兩個欄位
                  ●      將 "Score" 欄位內值大於等於 4 的轉成 1，其餘轉成 0 (1: positive 0: negative)
                  ●      將 "Text" 欄位內的文字利用分割符號切割
         1.2      去除停頓詞stop words ，可參考：
                  ●      sklearn.feature_extraction.text.CountVectorizer
                  ●      自訂stop words
         1.3      文字探勘前處理，將文字轉換成向量，請實作 tf-idf 及 word2vec 並進行比較，可參考：
                  ●      sklearn.feature_extraction.text.TfidfVectorizer
                  ●      Word2vec

2.          建模：使用 Random forest 進行分類

3.          評估模型：進行 k-fold cross-validation 並計算 k=4 的 Accuracy，上傳程式碼與報告

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv('/content/drive/MyDrive/碩一上課堂/1131_dataMining/dataMining/HW2/Reviews.csv')
test = pd.read_csv('/content/drive/MyDrive/碩一上課堂/1131_dataMining/dataMining/HW2/test.csv')

In [5]:
train = train.iloc[:10000]

# 將 "Score" 欄位中的值大於等於 4 的轉換成 1，其餘轉成 0
train['Sentiment'] = train['Score'].apply(lambda x: 1 if x >= 4 else 0)
train['Score']=train['Sentiment']

<ipython-input-5-0dae12baba85>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Sentiment'] = train['Score'].apply(lambda x: 1 if x >= 4 else 0)
<ipython-input-5-0dae12baba85>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Score']=train['Sentiment']


In [7]:
# 將 "Text" 欄位中的文字按照空格進行切割
train['Text_Split'] = train['Text'].apply(lambda x: x.split())

In [8]:
# 只選擇 Sentiment 和 Text 欄位
train = train[['Sentiment', 'Text_Split']]
# 查看篩選後的數據
train.head()

,Sentiment,Text_Split
0,1,"[I, have, bought, several, of, the, Vitality, ..."
1,0,"[Product, arrived, labeled, as, Jumbo, Salted,..."
2,1,"[This, is, a, confection, that, has, been, aro..."
3,0,"[If, you, are, looking, for, the, secret, ingr..."
4,1,"[Great, taffy, at, a, great, price., There, wa..."


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# 建立 CountVectorizer 並去除英文停頓詞
vectorizer = CountVectorizer(stop_words='english')

# 將 "Text_Split" 欄位中的詞語列表轉換回字串，並用空格連接
train['Text_Joined'] = train['Text_Split'].apply(lambda x: ' '.join(x))

# 將 "Text_Joined" 欄位進行詞頻向量化，並轉換成詞頻矩陣
train_text_matrix = vectorizer.fit_transform(train['Text_Joined'])

<ipython-input-9-1de998e7206b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Text_Joined'] = train['Text_Split'].apply(lambda x: ' '.join(x))


In [10]:
# 查看結果，顯示前幾個向量化的特徵名稱
vectorizer.get_feature_names_out()[:10]

array(['00', '000', '0003', '000kwh', '002', '008', '0100', '0174', '02',
       '03'], dtype=object)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 使用 TfidfVectorizer 將文字轉換成 TF-IDF 向量
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(train['Text_Joined'])

# 查看 TF-IDF 特徵名稱與矩陣形狀
print("TF-IDF 特徵名稱:", tfidf_vectorizer.get_feature_names_out()[:10])
print("TF-IDF 矩陣形狀:", tfidf_matrix.shape)

TF-IDF 特徵名稱: ['00' '000' '0003' '000kwh' '002' '008' '0100' '0174' '02' '03']
TF-IDF 矩陣形狀: (10000, 18497)


In [12]:
import gensim
from gensim.models import Word2Vec

# 將文本分詞，作為 Word2Vec 的輸入
sentences = [text.split() for text in train['Text_Joined']]

# 訓練 Word2Vec 模型
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

# 檢查詞向量
word_vectors = w2v_model.wv
print("單詞 'good' 的詞向量:", word_vectors['good'])

# 查看詞彙表中的單詞
print("詞彙表中前幾個單詞:", list(word_vectors.index_to_key)[:10])

單詞 'good' 的詞向量: [ 1.9225432  -0.47449127  0.19446209  0.8912177  -1.7635849  -1.735948
 -0.02128645  2.8026247  -0.53308845 -0.37741864  0.6276876  -0.334014
  1.3019202   1.0143217   0.5592164  -0.93458444 -0.17746797  0.00710296
 -0.5011382  -1.384002    1.9531538   0.78040665  0.07337439 -1.5564762
 -0.03785439 -0.30294925 -0.13875423 -1.4696052   0.9126311   0.5855332
 -1.450689    0.70107543  0.7886719  -1.1178389  -1.5856347   0.8806124
 -0.4583492  -0.2241123  -1.1303219  -0.21381584  0.55194485 -0.518169
  0.70356613 -0.16478164  1.1614431  -0.06038078 -0.993599   -1.1295825
  0.7546568   0.77918977 -0.5223206  -0.07687534 -0.5169344  -0.34535813
  1.6358333  -0.57674927  0.6031068   0.29386595  1.5168724  -0.96183836
  1.583746   -0.6042128   1.2860452  -0.89399606 -0.78255224  2.3356998
  0.3880542   0.6028807  -2.3409338   1.9738879   0.49397117  0.13006438
 -0.00387746 -0.6145722   0.33229202 -0.44167715  1.2782948  -0.6410463
 -0.95977116  1.0088003   0.9129078   1.0368747

## 透過RF分類

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# 使用 TfidfVectorizer 將文字轉換成 TF-IDF 向量
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(train['Text_Joined'])

# 建立隨機森林模型
rf_tfidf = RandomForestClassifier(n_estimators=100, random_state=42)

# 使用 k=4 進行交叉驗證
cv_scores_tfidf = cross_val_score(rf_tfidf, tfidf_matrix, train['Sentiment'], cv=4, scoring='accuracy')

# 輸出每次交叉驗證的準確率以及平均準確率
print("TF-IDF + 隨機森林 k=4 的準確率:", cv_scores_tfidf)
print("TF-IDF + 隨機森林 k=4 的平均準確率:", cv_scores_tfidf.mean())


TF-IDF + 隨機森林 k=4 的準確率: [0.7984 0.7984 0.7996 0.8052]
TF-IDF + 隨機森林 k=4 的平均準確率: 0.8004


In [14]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# 定義一個函數來將每個文本的詞向量取平均值
def average_word_vectors(words, model, num_features):
    feature_vec = np.zeros((num_features,), dtype="float32")
    nwords = 0
    index2word_set = set(model.wv.index_to_key)  # 確保詞彙表存在
    for word in words:
        if word in index2word_set:  # 檢查該單詞是否在詞彙表中
            nwords += 1
            feature_vec = np.add(feature_vec, model.wv[word])
    if nwords > 0:  # 避免除以0
        feature_vec = np.divide(feature_vec, nwords)
    return feature_vec

# 確保 'Text_Split' 欄位是已經分詞的文本
# sentences 應該是分詞過的句子的列表
sentences = train['Text_Split'].tolist()

# 訓練 Word2Vec 模型
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

# 將每個文本轉換為詞向量的平均值
train_word2vec = np.array([average_word_vectors(text, w2v_model, 100) for text in train['Text_Split']])

# 確保轉換後的詞向量是非空的
print(f"轉換後的詞向量形狀: {train_word2vec.shape}")

# 建立隨機森林模型
rf_w2v = RandomForestClassifier(n_estimators=100, random_state=42)

# 使用 k=4 進行交叉驗證
cv_scores_w2v = cross_val_score(rf_w2v, train_word2vec, train['Sentiment'], cv=4, scoring='accuracy')

# 輸出每次交叉驗證的準確率以及平均準確率
print("Word2Vec + 隨機森林 k=4 的準確率:", cv_scores_w2v)
print("Word2Vec + 隨機森林 k=4 的平均準確率:", cv_scores_w2v.mean())

轉換後的詞向量形狀: (10000, 100)
Word2Vec + 隨機森林 k=4 的準確率: [0.7756 0.772  0.7692 0.7564]
Word2Vec + 隨機森林 k=4 的平均準確率: 0.7683


In [15]:
# 使用已訓練好的 TfidfVectorizer 轉換 test 文本
test_tfidf_matrix = tfidf_vectorizer.transform(test['Text'])

In [16]:
# 將 "Text" 欄位中的文字按照空格進行切割
test['Text_Split'] = test['Text'].apply(lambda x: x.split())

# 將每個文本轉換為詞向量的平均值，使用已訓練的 Word2Vec 模型
test_word2vec = np.array([average_word_vectors(text, w2v_model, 100) for text in test['Text_Split']])

In [17]:
rf_tfidf.fit(tfidf_matrix, train['Sentiment'])
test_predictions_tfidf = rf_tfidf.predict(test_tfidf_matrix)

In [18]:
rf_w2v.fit(train_word2vec, train['Sentiment'])
# 使用隨機森林模型進行預測
test_predictions_w2v = rf_w2v.predict(test_word2vec)

In [19]:
# 為 test 資料生成一個從 1 開始的 Id 欄位
test['Id'] = range(1, len(test) + 1)  # 生成唯一的順序識別符

# 建立一個 DataFrame 並填入 TF-IDF 預測結果
submission_tfidf = pd.DataFrame({
    'ID': test['Id'],  # 使用順序作為唯一識別符
    'Score': test_predictions_tfidf  # 使用 TF-IDF 模型的預測結果
})

# 輸出結果至 CSV 文件，名稱為 tfidf_submission.csv
submission_tfidf.to_csv('/content/drive/MyDrive/碩一上課堂/1131_dataMining/dataMining/HW2/tfidf_submission.csv', index=False)

# 同樣的方法用於 Word2Vec 預測結果
submission_w2v = pd.DataFrame({
    'ID': test['Id'],  # 使用順序作為唯一識別符
    'Score': test_predictions_w2v  # 使用 Word2Vec 模型的預測結果
})

# 輸出結果至 CSV 文件，名稱為 w2v_submission.csv
submission_w2v.to_csv('/content/drive/MyDrive/碩一上課堂/1131_dataMining/dataMining/HW2/w2v_submission.csv', index=False)


## 透過XGboost分類


In [20]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score

# 初始化 XGBoost 模型
xgb_tfidf = xgb.XGBClassifier(n_estimators=100, random_state=42)
xgb_w2v = xgb.XGBClassifier(n_estimators=100, random_state=42)

# 使用 TF-IDF 特徵進行 XGBoost 訓練
xgb_tfidf.fit(tfidf_matrix, train['Sentiment'])

# 使用 Word2Vec 特徵進行 XGBoost 訓練
xgb_w2v.fit(train_word2vec, train['Sentiment'])

# 使用 TF-IDF 特徵進行預測
test_predictions_tfidf = xgb_tfidf.predict(test_tfidf_matrix)

# 使用 Word2Vec 特徵進行預測
test_predictions_w2v = xgb_w2v.predict(test_word2vec)

# 進行 k-fold 交叉驗證 (k=4) 並計算準確率
cv_scores_tfidf = cross_val_score(xgb_tfidf, tfidf_matrix, train['Sentiment'], cv=4, scoring='accuracy')
print("XGBoost (TF-IDF) k=4 的準確率:", cv_scores_tfidf)
print("XGBoost (TF-IDF) k=4 的平均準確率:", cv_scores_tfidf.mean())

cv_scores_w2v = cross_val_score(xgb_w2v, train_word2vec, train['Sentiment'], cv=4, scoring='accuracy')
print("XGBoost (Word2Vec) k=4 的準確率:", cv_scores_w2v)
print("XGBoost (Word2Vec) k=4 的平均準確率:", cv_scores_w2v.mean())


XGBoost (TF-IDF) k=4 的準確率: [0.824  0.8228 0.8292 0.8288]
XGBoost (TF-IDF) k=4 的平均準確率: 0.8262
XGBoost (Word2Vec) k=4 的準確率: [0.7764 0.768  0.7716 0.7596]
XGBoost (Word2Vec) k=4 的平均準確率: 0.7688999999999999


In [21]:
# 為 test 資料生成一個從 1 開始的 Id 欄位
test['Id'] = range(1, len(test) + 1)  # 生成唯一的順序識別符

# 建立一個 DataFrame 並填入 TF-IDF 預測結果
submission_tfidf = pd.DataFrame({
    'ID': test['Id'],  # 使用順序作為唯一識別符
    'Score': test_predictions_tfidf  # 使用 TF-IDF 模型的預測結果
})

# 輸出結果至 CSV 文件，名稱為 tfidf_submission.csv
submission_tfidf.to_csv('/content/drive/MyDrive/碩一上課堂/1131_dataMining/dataMining/HW2/tfidf_submission.csv', index=False)

# 同樣的方法用於 Word2Vec 預測結果
submission_w2v = pd.DataFrame({
    'ID': test['Id'],  # 使用順序作為唯一識別符
    'Score': test_predictions_w2v  # 使用 Word2Vec 模型的預測結果
})

# 輸出結果至 CSV 文件，名稱為 w2v_submission.csv
submission_w2v.to_csv('/content/drive/MyDrive/碩一上課堂/1131_dataMining/dataMining/HW2/w2v_submission.csv', index=False)
